In [9]:
import os
import glob

import numpy as np
import cv2
import lightgbm as lgbm

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print('Import OK!')

Import OK!


Read images and extract features  
    Use canny algorithm to extract edges from images. Flatten the images into 1D vectors

In [2]:
def loadDataFromDir(**dirNames):
    #Image dimensions
    imH = 80
    imW = 80
    
    #Thresholding for canny edge detection
    minVal = 50
    maxVal = 250


    X = np.array([])
    y = []
    
    for label, dirName in dirNames.items():
        for f in glob.glob(dirName + '/*.png'):
            img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
            img = img[:imH+1, :imW+1] #crop
            edges = cv2.Canny(img, minVal, maxVal) #extract edges
            edges = edges.reshape(np.prod(edges.shape)) #flatten
            
            #append
            if len(X) == 0:
                X = edges
            else:
                X = np.vstack((X, edges))
            
            assert label in ('target', 'nonTarget')
            if(label == 'target'):
                y.append(1)
            elif(label == 'nonTarget'):
                y.append(0)

    return X, y

In [3]:
targetTrainDir = "target_train"
nonTargetTrainDir = "non_target_train"

#Read target data
XTrain, yTrain = loadDataFromDir(target=targetTrainDir, nonTarget=nonTargetTrainDir)
    
print('Feature extraction OK!')

Feature extraction OK!


Train the model  
Use Light Gradient Boosting Machine (use scikit-learn api of ligthgbm)

In [23]:
gbmClassifier = lgbm.LGBMModel(objective='binary', random_state=42)
gbmClassifier.fit(XTrain, yTrain)

print('Training OK!')

Training OK!


Evaluate the model

In [27]:
targetTestDir = "target_dev"
nonTargetTestDir = "non_target_dev"

XTest, yTest = loadDataFromDir(target=targetTestDir, nonTarget=nonTargetTestDir)

predictions = gbmClassifier.predict(XTest)
predictions = np.round(predictions)
predictions = predictions.astype(int)

print(predictions)
print(confusion_matrix(yTest, predictions)) #[[TrueNegatives, TruePositives][FalseNegative, FalsePositives]]
print(accuracy_score(yTest, predictions))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[59  1]
 [10  0]]
0.8428571428571429
